In [93]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from doubleml.data import DoubleMLPanelData
from doubleml.did import DoubleMLDIDMulti
import os

In [94]:
dta_study = pd.read_stata("CashBail.dta")

dta_study["EligibleOffense"] = dta_study["EligibleOffense"].astype("float")
dta_study.loc[dta_study["EligibleOffense"] == 0, "EligibleOffense"] = np.inf

In [95]:
dta_study_clean = dta_study[
    [
        "docketnumber_anon",
        "EligibleOffense",
        "Post",
        "hasfta",
        "defendantageatarrest",
        "male",
        "defendantisblack",
        "Hisp",
        "prior",
        "prior_FTA",
        "felony",
        "nb_charges",
    ]
]

In [96]:
np.random.seed(10)
all_results_list = [] 

outcomes = ["ROR", "Jail3days", "hasfta", "Rec6mo"]

for y in outcomes:
    print(f"Now processing: {y}...")
    
    dml_data = DoubleMLPanelData(
        data=dta_study,
        y_col=y,
        d_cols="EligibleOffense",
        t_col="Post",
        x_cols=[
            "defendantageatarrest", "male", "defendantisblack", "Hisp", "prior", "prior_FTA", "felony", "nb_charges"],
        id_col="docketnumber_anon"
    )

    dml_obj = DoubleMLDIDMulti(
        obj_dml_data=dml_data,
        ml_g=RandomForestRegressor(n_estimators=100, max_depth=5, random_state=10),
        ml_m=RandomForestClassifier(n_estimators=100, max_depth=5, random_state=10),
        control_group="never_treated",
        panel=False,
        n_folds=5
    )


    dml_obj.fit(store_predictions=True)
    
    temp_summary = dml_obj.summary.copy()
    temp_summary.insert(0, 'Outcome_Var', y)
    
    all_results_list.append(temp_summary)

final_summary_table = pd.concat(all_results_list, ignore_index=True)


Now processing: ROR...
Now processing: Jail3days...
Now processing: hasfta...
Now processing: Rec6mo...


In [97]:
final_summary_table

,Outcome_Var,coef,std err,t,P>|t|,2.5 %,97.5 %
0,ROR,0.081164,0.013222,6.138651,8.322512e-10,0.055250,0.107079
1,Jail3days,0.008401,0.012542,0.669870,5.029408e-01,-0.016180,0.032983
2,hasfta,-0.022185,0.011929,-1.859815,6.291165e-02,-0.045565,0.001195
3,Rec6mo,-0.019990,0.011882,-1.682413,9.248884e-02,-0.043278,0.003298
